In [ ]:
!pip install transformers datasets tokenizers seqeval -q
!pip install accelerate -U

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 542.0/542.0 kB 6.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.6/43.6 kB 4.9 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 7.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 15.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 15.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 401.2/401.2 kB 21.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 302.6/302.6 kB 4.6 MB/s eta 0:00:00
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (23.7 MB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (823 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (14.1 MB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl (731.7 MB)
  Using cached nvidia_cublas_cu12-1

In [ ]:
from sklearn.model_selection import train_test_split
import datasets
import pandas as pd
import torch
from transformers import pipeline
from sklearn.preprocessing import LabelEncoder
from datasets import DatasetDict, Dataset
import numpy as np
from transformers import BertTokenizerFast
from transformers import DataCollatorForTokenClassification
from transformers import AutoModelForTokenClassification
from transformers import Trainer, TrainingArguments
from sklearn.metrics import accuracy_score

In [ ]:
filename = "Full-Balanced version - Numeric one.xlsx"

# Read the data from the Excel file into a Pandas DataFrame
df = pd.read_excel(filename)

# Save the DataFrame to another format (optional)
df.to_csv("output.csv")  # Save as CSV
# Load data from your CSV/file (replace with your loading logic)
data = pd.read_csv("output.csv", encoding="utf-8")
data.head()

,Unnamed: 0,old indices,sentence,class,label
0,0,3,ابحث عن مجلد image processing,بحث,"{ ""نوع العنصر"":""مجلد"",""image processing"":""اسم ..."
1,1,6,من فضلك ابحث عن مجلد album 1,بحث,"{ ""نوع العنصر"":""مجلد"",""album 1"":""اسم العنصر""}"
2,2,9,من فضلك ابحث عن مجلد math 1,بحث,"{ ""نوع العنصر"":""مجلد"",""math 1"":""اسم العنصر""}"
3,3,10,ابحث في الحاسوب عن مجلد grades,بحث,"{ ""نوع العنصر"":""مجلد"",""grades"":""اسم العنصر""}"
4,4,12,ابحث في الحاسوب عن مجلد results,بحث,"{ ""نوع العنصر"":""مجلد"",""results"":""اسم العنصر""}"


In [ ]:
data.drop('class', axis=1, inplace=True)
data.drop('old indices', axis=1, inplace=True)


In [ ]:
data.head()

,Unnamed: 0,sentence,label
0,0,ابحث عن مجلد image processing,"{ ""نوع العنصر"":""مجلد"",""image processing"":""اسم ..."
1,1,من فضلك ابحث عن مجلد album 1,"{ ""نوع العنصر"":""مجلد"",""album 1"":""اسم العنصر""}"
2,2,من فضلك ابحث عن مجلد math 1,"{ ""نوع العنصر"":""مجلد"",""math 1"":""اسم العنصر""}"
3,3,ابحث في الحاسوب عن مجلد grades,"{ ""نوع العنصر"":""مجلد"",""grades"":""اسم العنصر""}"
4,4,ابحث في الحاسوب عن مجلد results,"{ ""نوع العنصر"":""مجلد"",""results"":""اسم العنصر""}"


In [ ]:
new_data = []
class1 = []
class2 = []
class3 = []
class4 = []
class5 = []
class6 = []

for i in range(272):
  class1.append(data.iloc[i])

for i in range(272, 544):
  class2.append(data.iloc[i])

for i in range(544, 740):
  class3.append(data.iloc[i])

for i in range(740, 1012):
  class4.append(data.iloc[i])

for i in range(1012, 1208):
  class5.append(data.iloc[i])

for i in range(1208, 1469):
  class6.append(data.iloc[i])

for i in range(1469, 1534):
  class5.append(data.iloc[i])

for i in range(1534, 1599):
  class3.append(data.iloc[i])

for i in range(1599, 1690):
  class4.append(data.iloc[i])

for i in range(1690, 1780):
  class2.append(data.iloc[i])

for i in range(1780, 1872):
  class1.append(data.iloc[i])

i = 0
while len(new_data) != 1872:
  if i < len(class1):
    new_data.append(class1[i])

  if i < len(class2):
    new_data.append(class2[i])

  if i < len(class3):
    new_data.append(class3[i])

  if i < len(class4):
    new_data.append(class4[i])

  if i < len(class5):
    new_data.append(class5[i])

  if i < len(class6):
    new_data.append(class6[i])

  i += 1

new_data = pd.DataFrame(new_data, columns=['sentence', 'label'])
new_data

,sentence,label
0,ابحث عن مجلد image processing,"{ ""نوع العنصر"":""مجلد"",""image processing"":""اسم ..."
272,في تمام 1 قم بغلق الجهاز,"{ ""اسم العنصر"":""الجهاز"",""وقت"":""1""}"
544,انتظر ساعتين ثم اعد تشغيل الجهاز,"{ ""اسم العنصر"":""الجهاز"",""فترة"":""ساعتين""}"
740,اضبط المنبه ليصدر صوت بعد ساعة,"{""فترة"":""ساعة"",""اسم العنصر"":""المنبه""}"
1012,من فضلك بعد ساعة و21 دقيقة ضع الجهاز في وضع ا...,"{""فترة"":""ساعة و21 دقيقة""}"
...,...,...
1779,في تمام الساعة 3 قم بغلق الجهاز,"{ ""اسم العنصر"":""الجهاز"",""وقت"":""الساعة 3""}"
1688,قم بضبط المؤقت لمدة 62 دقيقة,"{""فترة"":""62 دقيقة"",""اسم العنصر"":""المؤقت""}"
1870,ابحث عن مجلد ملفات الألعاب الإلكترونية الترفيهية,"{""اسم العنصر"":""ملفات الألعاب الإلكترونية الترف..."
1689,قم بضبط المؤقت لمدة 72 دقيقة,"{""فترة"":""72 دقيقة"",""اسم العنصر"":""المؤقت""}"


In [ ]:
len(new_data)

1872

In [ ]:
def process_data(df):

    processed_data = []

    for sentence_index, row in df.iterrows():
      # print(row[1])
      sentence = row[0].strip()  # Assuming the first column contains the sentence
      words = sentence.split()
      words_list = []
      labels_list = []

      # Extract labels from the existing label column (assuming specific format)
      label_dict = eval(row[1]) if pd.notna(row[1]) else {}  # Handle potential missing values
      right_dic = {}
      # print(label_dict)
      # print(sentence_index)
      for label_word, label_value in label_dict.items():
        # print(f'label_word : {label_word}')
        # print(f'label_value : {label_value}')

        if label_word in sentence and label_value not in sentence:
          right_dic[label_value] = label_word

        elif label_value in sentence:
          right_dic[label_word] = label_value

      # print(f'right_dic : {right_dic}')
      for word_index, word in enumerate(words):
          # print(f'Word : {word}')
          label = 'O'

          for label_word, label_value in right_dic.items():
            if word in label_value:
                label = label_word
                break  # Exit the loop once a match is found

          words_list.append(word)
          labels_list.append(label.strip())

      processed_data.append({
          "sentence_id": sentence_index,
          "words": words_list,
          "labels": labels_list
      })

    return pd.DataFrame(processed_data)

# Replace 'your_data.csv' with the actual path to your CSV file
processed_df = process_data(new_data)

print(processed_df.iloc[0])

sentence_id                                             0
words                 [ابحث, عن, مجلد, image, processing]
labels         [O, O, نوع العنصر, اسم العنصر, اسم العنصر]
Name: 0, dtype: object


In [ ]:
from sklearn.preprocessing import LabelEncoder

# Flatten the list of lists
all_labels = [label for sublist in processed_df['labels'] for label in sublist]

# Initialize a LabelEncoder object
label_encoder = LabelEncoder()

# Fit the LabelEncoder to your labels to create the mapping
label_encoder.fit(all_labels)

# Transform your labels using the fitted LabelEncoder
processed_df['labels'] = processed_df['labels'].apply(lambda x: label_encoder.transform(x))

# Get the unique integer labels
unique_labels = label_encoder.classes_

# Print the mapping
print("Label Encoding Mapping:")
for label, encoded_label in zip(unique_labels, range(len(unique_labels))):
    print(f"{label} -> {encoded_label}")

# Print the encoded labels
print("Encoded Labels:")
print(processed_df['labels'])

Label Encoding Mapping:
O -> 0
اسم العنصر -> 1
الملاحظة -> 2
فترة -> 3
نوع العنصر -> 4
وقت -> 5
Encoded Labels:
0                         [0, 0, 4, 1, 1]
1                      [0, 0, 5, 0, 0, 1]
2                      [0, 3, 0, 0, 0, 1]
3                      [0, 1, 0, 0, 0, 3]
4       [0, 0, 0, 3, 3, 3, 0, 0, 0, 0, 0]
                      ...                
1867                [0, 0, 5, 5, 0, 0, 1]
1868                   [0, 0, 1, 0, 3, 3]
1869                [0, 0, 4, 1, 1, 1, 1]
1870                   [0, 0, 1, 0, 3, 3]
1871                [0, 0, 4, 1, 1, 1, 1]
Name: labels, Length: 1872, dtype: object


In [ ]:
X = processed_df[["sentence_id", "words"]]
Y = processed_df["labels"]
X

,sentence_id,words
0,0,"[ابحث, عن, مجلد, image, processing]"
1,272,"[في, تمام, 1, قم, بغلق, الجهاز]"
2,544,"[انتظر, ساعتين, ثم, اعد, تشغيل, الجهاز]"
3,740,"[اضبط, المنبه, ليصدر, صوت, بعد, ساعة]"
4,1012,"[من, فضلك, بعد, ساعة, و21, دقيقة, ضع, الجهاز, ..."
...,...,...
1867,1779,"[في, تمام, الساعة, 3, قم, بغلق, الجهاز]"
1868,1688,"[قم, بضبط, المؤقت, لمدة, 62, دقيقة]"
1869,1870,"[ابحث, عن, مجلد, ملفات, الألعاب, الإلكترونية, ..."
1870,1689,"[قم, بضبط, المؤقت, لمدة, 72, دقيقة]"


In [ ]:
x_train, x_test, y_train, y_test = train_test_split(X,Y, test_size =0.2)

In [ ]:
x_train.head()

,sentence_id,words
1074,179,"[اعثر, لي, على, ملف, النصائح, لتحسين, الاداء, ..."
1371,968,"[يرجي, تعيين, المؤقت, لمدة, 85, ساعة, و9, دقائق]"
1265,1418,"[اكتب, في, مذكراتي, الاتي, انتظر, عشر, دقائق, ..."
1836,1859,"[استخرج, مجلد, المشاريع, الفنية, للعملاء]"
1263,950,"[يرجي, ضبط, المؤقت, لمدة, 98, ساعة, و30, دقيقة]"


In [ ]:
len(x_test), len(x_train)

(375, 1497)

In [ ]:
train_dataset = Dataset.from_dict({
    "id": range(1497),
    "tokens": x_train['words'],
    "ner_tags": y_train
})

test_dataset = Dataset.from_dict({
    "id": range(375),
    "tokens": x_test['words'],
    "ner_tags": y_test
})
train_dataset

Dataset({
    features: ['id', 'tokens', 'ner_tags'],
    num_rows: 1497
})

In [ ]:
allData = DatasetDict({
    'train': train_dataset,
    'test' : test_dataset
})
allData

DatasetDict({
    train: Dataset({
        features: ['id', 'tokens', 'ner_tags'],
        num_rows: 1497
    })
    test: Dataset({
        features: ['id', 'tokens', 'ner_tags'],
        num_rows: 375
    })
})

In [ ]:
tokenizer = BertTokenizerFast.from_pretrained("aubmindlab/bert-base-arabertv2")


/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/611 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/720k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.31M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/384 [00:00<?, ?B/s]

In [ ]:
def tokenize_and_align_labels(examples, label_all_tokens=True):
    """
    Function to tokenize and align labels with respect to the tokens. This function is specifically designed for
    Named Entity Recognition (NER) tasks where alignment of the labels is necessary after tokenization.

    Parameters:
    examples (dict): A dictionary containing the tokens and the corresponding NER tags.
                     - "tokens": list of words in a sentence.
                     - "ner_tags": list of corresponding entity tags for each word.

    label_all_tokens (bool): A flag to indicate whether all tokens should have labels.
                             If False, only the first token of a word will have a label,
                             the other tokens (subwords) corresponding to the same word will be assigned -100.

    Returns:
    tokenized_inputs (dict): A dictionary containing the tokenized inputs and the corresponding labels aligned with the tokens.
    """

    tokenized_inputs = tokenizer(examples["tokens"], truncation=True, is_split_into_words=True)
    labels = []
    for i, label in enumerate(examples["ner_tags"]):
        word_ids = tokenized_inputs.word_ids(batch_index=i)
        # word_ids() => Return a list mapping the tokens
        # to their actual word in the initial sentence.
        # It Returns a list indicating the word corresponding to each token.
        previous_word_idx = None
        label_ids = []
        # Special tokens like `<s>` and `<\s>` are originally mapped to None
        # We need to set the label to -100 so they are automatically ignored in the loss function.
        for word_idx in word_ids:
            if word_idx is None:
                # set –100 as the label for these special tokens
                label_ids.append(-100)
            # For the other tokens in a word, we set the label to either the current label or -100, depending on
            # the label_all_tokens flag.
            elif word_idx != previous_word_idx:
                # if current word_idx is != prev then its the most regular case
                # and add the corresponding token
                label_ids.append(label[word_idx])
            else:
                # to take care of sub-words which have the same word_idx
                # set -100 as well for them, but only if label_all_tokens == False
                label_ids.append(label[word_idx] if label_all_tokens else -100)
                # mask the subword representations after the first subword

            previous_word_idx = word_idx
        labels.append(label_ids)
    tokenized_inputs["labels"] = labels
    return tokenized_inputs

In [ ]:
tokenized_datasets = allData.map(tokenize_and_align_labels, batched=True)

Map:   0%|          | 0/1497 [00:00<?, ? examples/s]

Map:   0%|          | 0/375 [00:00<?, ? examples/s]

In [ ]:
model = AutoModelForTokenClassification.from_pretrained("aubmindlab/bert-base-arabertv2", num_labels=6)

model.safetensors:   0%|          | 0.00/543M [00:00<?, ?B/s]

Some weights of BertForTokenClassification were not initialized from the model checkpoint at aubmindlab/bert-base-arabertv2 and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
data_collator = DataCollatorForTokenClassification(tokenizer)

In [ ]:
metric = datasets.load_metric("seqeval")


<ipython-input-24-b144b02b338d>:1: FutureWarning: load_metric is deprecated and will be removed in the next major version of datasets. Use 'evaluate.load' instead, from the new library 🤗 Evaluate: https://huggingface.co/docs/evaluate
  metric = datasets.load_metric("seqeval")
/usr/local/lib/python3.10/dist-packages/datasets/load.py:759: FutureWarning: The repository for seqeval contains custom code which must be executed to correctly load the metric. You can inspect the repository content at https://raw.githubusercontent.com/huggingface/datasets/2.19.1/metrics/seqeval/seqeval.py
You can avoid this message in future by passing the argument `trust_remote_code=True`.
Passing `trust_remote_code=True` will be mandatory to load this metric from the next major release of `datasets`.
  warnings.warn(


In [ ]:
allData

DatasetDict({
    train: Dataset({
        features: ['id', 'tokens', 'ner_tags'],
        num_rows: 1497
    })
    test: Dataset({
        features: ['id', 'tokens', 'ner_tags'],
        num_rows: 375
    })
})

In [ ]:
unique_labels

array(['O', 'اسم العنصر', 'الملاحظة', 'فترة', 'نوع العنصر', 'وقت'],
      dtype='<U10')

In [ ]:
def compute_metrics(eval_preds):
    """
    Function to compute the evaluation metrics for Named Entity Recognition (NER) tasks.
    The function computes precision, recall, F1 score and accuracy.

    Parameters:
    eval_preds (tuple): A tuple containing the predicted logits and the true labels.

    Returns:
    A dictionary containing the precision, recall, F1 score and accuracy.
    """
    pred_logits, labels = eval_preds

    pred_logits = np.argmax(pred_logits, axis=2)
    # the logits and the probabilities are in the same order,
    # so we don’t need to apply the softmax

    # We remove all the values where the label is -100
    predictions = [
        [unique_labels[eval_preds] for (eval_preds, l) in zip(prediction, label) if l != -100]
        for prediction, label in zip(pred_logits, labels)
    ]

    true_labels = [
      [unique_labels[l] for (eval_preds, l) in zip(prediction, label) if l != -100]
       for prediction, label in zip(pred_logits, labels)
   ]

    results = metric.compute(predictions=predictions, references=true_labels)
    return {
   "precision": results["overall_precision"],
   "recall": results["overall_recall"],
   "f1": results["overall_f1"],
  "accuracy": results["overall_accuracy"],
  }

In [ ]:
from transformers import TrainingArguments, Trainer
from transformers import get_linear_schedule_with_warmup

args = TrainingArguments(
  "test-ner",
  evaluation_strategy="steps",
  learning_rate=2e-5,  # Start with a lower learning rate
  per_device_train_batch_size=16,
  per_device_eval_batch_size=32,
  num_train_epochs=3,
  weight_decay=0.01,
  logging_strategy='steps',
  logging_steps=50,
  eval_steps=50,
  save_strategy="steps",
  # Learning rate scheduling arguments
  warmup_steps=100,  # Warmup steps for gradual learning rate increase
  load_best_model_at_end=True
)

In [ ]:
trainer = Trainer(
    model,
    args,
   train_dataset=tokenized_datasets["train"],
   eval_dataset=tokenized_datasets["test"],
   data_collator=data_collator,
   tokenizer=tokenizer,
   compute_metrics=compute_metrics
)

In [ ]:
trainer.evaluate()

/usr/local/lib/python3.10/dist-packages/seqeval/metrics/sequence_labeling.py:171: UserWarning: نوع العنصر seems not to be NE tag.
  warnings.warn('{} seems not to be NE tag.'.format(chunk))
/usr/local/lib/python3.10/dist-packages/seqeval/metrics/sequence_labeling.py:171: UserWarning: اسم العنصر seems not to be NE tag.
  warnings.warn('{} seems not to be NE tag.'.format(chunk))
/usr/local/lib/python3.10/dist-packages/seqeval/metrics/sequence_labeling.py:171: UserWarning: وقت seems not to be NE tag.
  warnings.warn('{} seems not to be NE tag.'.format(chunk))
/usr/local/lib/python3.10/dist-packages/seqeval/metrics/sequence_labeling.py:171: UserWarning: فترة seems not to be NE tag.
  warnings.warn('{} seems not to be NE tag.'.format(chunk))
/usr/local/lib/python3.10/dist-packages/seqeval/metrics/sequence_labeling.py:171: UserWarning: الملاحظة seems not to be NE tag.
  warnings.warn('{} seems not to be NE tag.'.format(chunk))


{'eval_loss': 1.6334744691848755,
 'eval_precision': 0.00696594427244582,
 'eval_recall': 0.01393188854489164,
 'eval_f1': 0.009287925696594427,
 'eval_accuracy': 0.4374747882210569,
 'eval_runtime': 2.8195,
 'eval_samples_per_second': 133.003,
 'eval_steps_per_second': 4.256}

In [ ]:
trainer.train()

Step,Training Loss,Validation Loss,Precision,Recall,F1,Accuracy
50,1.421700,0.996923,0.198425,0.195046,0.196721,0.708754
100,0.641900,0.245536,0.682770,0.656347,0.669298,0.918112
150,0.209400,0.125931,0.943750,0.934985,0.939347,0.971965
200,0.147700,0.112984,0.955247,0.958204,0.956723,0.973578
250,0.116900,0.102377,0.968304,0.945820,0.956930,0.976200


/usr/local/lib/python3.10/dist-packages/seqeval/metrics/v1.py:57: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


TrainOutput(global_step=282, training_loss=0.460013747215271, metrics={'train_runtime': 50.5514, 'train_samples_per_second': 88.84, 'train_steps_per_second': 5.578, 'total_flos': 68054445442980.0, 'train_loss': 0.460013747215271, 'epoch': 3.0})

In [ ]:
trainer.evaluate()

{'eval_loss': 0.10309506952762604,
 'eval_precision': 0.9639498432601881,
 'eval_recall': 0.9520123839009288,
 'eval_f1': 0.9579439252336448,
 'eval_accuracy': 0.9762000806776926,
 'eval_runtime': 1.1273,
 'eval_samples_per_second': 332.666,
 'eval_steps_per_second': 10.645,
 'epoch': 3.0}

In [ ]:
model.save_pretrained("ner_model")


In [ ]:
tokenizer.save_pretrained("tokenizer")


('tokenizer/tokenizer_config.json',
 'tokenizer/special_tokens_map.json',
 'tokenizer/vocab.txt',
 'tokenizer/added_tokens.json',
 'tokenizer/tokenizer.json')

In [ ]:
id2label = {
    '0': 'O',
    '1': 'اسم العنصر',
    '2' : 'الملاحظة',
    '3' : 'فترة',
    '4' : 'نوع العنصر',
    '5': 'وقت'
}
label2id = {
    'O' : '0',
    'اسم العنصر' : '1',
    'الملاحظة' : '2',
    'فترة' : '3',
    'نوع العنصر' : '4',
    'وقت': '5'
}
id2label['0']

'O'

In [ ]:
import json

In [ ]:
config = json.load(open("ner_model/config.json"))
config["id2label"] = id2label
config["label2id"] = label2id
json.dump(config, open("ner_model/config.json","w"))

In [ ]:
from google.colab import drive
drive.mount('/content/drive')


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
import shutil

# Specify the source directory (Colab)
source_dir = '/content/ner_model'

# Specify the destination directory (Google Drive)
dest_dir = '/content/drive/MyDrive/Model'

# Copy the folder from source to destination
shutil.copytree(source_dir, dest_dir)
shutil.copytree('/content/tokenizer', '/content/drive/MyDrive/ModelTokenizer')

'/content/drive/MyDrive/ModelTokenizer'